# モデル実験開発 LightGBM

---

## 1. はじめに

このノートブックでは、SageMakerのトレーニングジョブを使用して、機械学習モデル（LightGBM）の実験開発を行うためのサンプルを用意しています。[こちらの AWS ブログ](https://aws.amazon.com/jp/blogs/news/associating-prediction-results-with-input-data-using-amazon-sagemaker-batch-transform/) の内容を元にしているため、まずはブログを読んで全体の流れを把握してからノートブックを実行していくのがおすすめです。

### 参考URL

Amazon SageMaker Python SDKのドキュメント    
- https://sagemaker.readthedocs.io/en/stable/index.html

SageMaker XGBoostのコンテナGithub
- https://github.com/aws/sagemaker-xgboost-container

英語のSageMakerサンプル集
- https://github.com/aws/amazon-sagemaker-examples

日本語のSageMakerサンプル集
- https://github.com/aws-samples/aws-ml-jp

## 2.セットアップ

まず、このノートブックインスタンスに付与されている IAM role を `get_execution_role()` から取得しましょう。後ほど、SageMaker の学習やホスティングを行いますが、そこで IAM role が必要になります。そこで、ノートブックインスタンスの IAM role を、学習やホスティングでも利用します。
通常、role を取得するためにはAWS SDKを利用した数行のコードを書く必要があります。ここでは `get_execution_role()` のみで role を取得可能です。SageMaker Python SDK は、データサイエンティストが機械学習以外のコードを簡潔に済ませるために、このような関数を提供しています。


In [ ]:
import sys

!{sys.executable} -m pip install --upgrade pip
!{sys.executable} -m pip install --upgrade sagemaker 'pandas>=1.0.5' shap lightgbm　
!{sys.executable} -m pip install --upgrade 'scikit-learn>=0.24.0,<1.0.0'

In [ ]:
# Define IAM role
import boto3
import re
from sagemaker import get_execution_role
import sagemaker

role = get_execution_role()
region = boto3.Session().region_name

sagemaker_session = sagemaker.Session()

bucket_name=sagemaker_session.default_bucket()
user_name = 'demo' # 同一アカウント内で重複した名前を防ぐためのものです
ver = 'v01'

以降で利用するライブラリをここで読み込んでおきます。

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
import os
import pickle as pkl
import sys
import time
import json
from IPython.display import display
from time import strftime, gmtime


print('Current SageMaker Python SDK Version ={0}'.format(sagemaker.__version__))

In [ ]:
# ローカルにディレクトリ作成
SRC_DIR = f'./src_lgb'
if not os.path.exists(SRC_DIR):
    os.makedirs(SRC_DIR)

---
## 3.データ

このNotebookではUCIリポジトリにある[Census-Income (KDD) Data Set](https://archive.ics.uci.edu/ml/datasets/Census-Income+%28KDD%29)を使用します。

In [ ]:
INPUT_DIR = './input/'
if not os.path.exists(INPUT_DIR):
    os.makedirs(INPUT_DIR)

In [ ]:
# データのダウンロード
s3 = boto3.client('s3')
input_data = 's3://sagemaker-sample-data-{}/processing/census/census-income.csv'.format(region)
!aws s3 cp $input_data ./input/

In [ ]:
# データの読み込みとターゲット変数の変換
df = pd.read_csv('./input/census-income.csv')
print(df.income.value_counts())
df['income'] = np.where(df['income'] == ' 50000+.', 1, 0) #ターゲット変数の(0, 1)変換
print(df.income.value_counts())

df.head()

データには94年と95年のものが含まれているため、94年は開発用、95年はテストおよび再学習・推論パイプライン作成に使用します。

In [ ]:
df_94 = df[df['year'] == 94].reset_index(drop=True)
df_95 = df[df['year'] == 95].reset_index(drop=True)

del df_94['year'], df_95['year']

print(df_94.shape)
print(df_94.income.value_counts())
print(df_95.shape)
print(df_95.income.value_counts())

## 4.EDA

データの統計量や相関を確認します。

**このセクションではSageMakerの機能は使用していません。pythonのライブラリを使用しています。**

In [ ]:
# Frequency tables for each categorical feature
for column in df_94.select_dtypes(include=['object']).columns:
    display(pd.crosstab(index=df_94[column], columns='% observations', normalize='columns'))

# Histograms for each numeric features
display(df_94.describe())
%matplotlib inline
hist = df_94.hist(bins=30, sharey=True, figsize=(10, 10))

In [ ]:
display(df_94.corr())
pd.plotting.scatter_matrix(df_94, figsize=(12, 12))
plt.show()

## 5.データの加工とS3へのアップロード

LightGBMのトレーニングジョブが適用できるようにデータを加工します。

In [ ]:
df_95 = pd.concat([df_95['income'], df_95.drop(['income'], axis=1)], axis=1)
df_95_raw = df_95.copy() # 再学習・推論パイプライン用
df_95.head()

In [ ]:
# ここで使用する学習用スクリプトはターゲット列の位置に制限はありませんが、取り回しのしやすさから0列目にしています
df_94 = pd.concat([df_94['income'], df_94.drop(['income'], axis=1)], axis=1)
df_94.head()

In [ ]:
#  開発用データを学習・検証データへと分割します。
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(df_94, test_size=0.1, shuffle=True, random_state=1, stratify=df_94.income)

print(train_df.income.value_counts())
print(val_df.income.value_counts())

In [ ]:
# カテゴリー列の抽出
categorical_columns = [x for x in df_94.columns if df_94[x].dtypes == 'object']
categorical_columns

In [ ]:
# カテゴリー列を数値へ変換します。
from sklearn.preprocessing import OrdinalEncoder

enc = OrdinalEncoder(
    handle_unknown = 'use_encoded_value', # sklearn 0.24以降
    unknown_value = -9999, # sklearn 0.24以降
)

# 欠損を文字列Noneへ変更
train_df.loc[:, categorical_columns] = train_df[categorical_columns].fillna('None')
val_df.loc[:, categorical_columns] = val_df[categorical_columns].fillna('None')
df_95.loc[:, categorical_columns] = df_95[categorical_columns].fillna('None')

enc.fit(train_df[categorical_columns])
pkl.dump(enc, open('preprocessor', 'wb')) # 推論パイプライン用にdumpする
#enc = pkl.load(open('preprocessor', 'rb'))

train_df.loc[:, categorical_columns] = enc.transform(train_df[categorical_columns])
val_df.loc[:, categorical_columns] = enc.transform(val_df[categorical_columns])
df_95.loc[:, categorical_columns] = enc.transform(df_95[categorical_columns])

In [ ]:
train_df['class of worker']

In [ ]:
train_df.head()

In [ ]:
val_df.head()

In [ ]:
df_95.head() # バッチ推論の動作確認用データ

In [ ]:
df_95_raw.head() # 推論・再学習パイプライン用

データをローカル（Notebookインスタンス）に出力して、S3へアップロードします。

In [ ]:
# ローカルにディレクトリ作成
DATASET_DIR = f'./dataset/{ver}/'
if not os.path.exists(DATASET_DIR):
    os.makedirs(DATASET_DIR)

**このあとの 5.トレーニングジョブの実行 のところで作成する LightGBMの学習スクリプト（変更可能）がヘッダーありのファイルを入力とする**よう記述しているため、学習・検証用ファイルはヘッダありで書き出します。一方、（バッチ推論時のフィルタリングの取り回しの関係で）推論テスト用のファイルはヘッダなしでファイルを書き出します。

In [ ]:
# ローカルにファイル書き出し
train_df.to_csv(DATASET_DIR+'train.csv', header=True, index=False)
val_df.to_csv(DATASET_DIR+'validation.csv', header=True, index=False)
df_95.to_csv(DATASET_DIR+'test.csv', header=False, index=False) # バッチ推論テスト

#df_95_raw.to_csv('new_train_data.csv', header=True, index=False) # 再学習パイプライン用
#del df_95_raw['income']
#df_95_raw.to_csv('20220415.csv', header=True, index=False) # 推論パイプライン用

In [ ]:
# S3へのデータアップロード
key = f'dataset/{ver}'
train_prefix = 'train'
val_prefix = 'validation'

boto3.Session().resource('s3').Bucket(bucket_name).Object(f'{key}/{train_prefix}.csv').upload_file(f'{key}/{train_prefix}.csv')
s3_input_train = f's3://{bucket_name}/{key}/{train_prefix}.csv'
print('Done writing to {}'.format(s3_input_train))

boto3.Session().resource('s3').Bucket(bucket_name).Object(f'{key}/{val_prefix}.csv').upload_file(f'{key}/{val_prefix}.csv')
s3_input_validation = f's3://{bucket_name}/{key}/{val_prefix}.csv'
print('Done writing to {}'.format(s3_input_validation))

In [ ]:
# データがcsv形式と想定して学習スクリプトを書いているので、S3へアップロードしたデータがcsv形式であると指定
from sagemaker.inputs import TrainingInput

content_type = 'text/csv'

train_input = TrainingInput(s3_input_train, content_type=content_type)
validation_input = TrainingInput(s3_input_validation, content_type=content_type)

## 6.トレーニングジョブの実行

以下のセルでスクリプトを書き出します。    
`model_fn()`に加えて、`input_fn()`、`predict_fn()`がXGBoostのサンプルにはなかった関数となり、推論時に使用される関数です。    
XGBoostの場合はデフォルト関数そのままで動作しますが、LightGBMでは動作しない部分もあるため、新たに定義しています。

In [ ]:
%%writefile ./src_lgb/lightgbm_train.py

import argparse
import logging
import os
import pickle as pkl
from io import StringIO

import numpy as np
import pandas as pd

import lightgbm as lgb


# バッチ推論用関数
def model_fn(model_dir):
    '''モデルを読み込む関数'''
    model_file = 'lgb-model'
    booster = lgb.Booster(model_file=os.path.join(model_dir, model_file))
    return booster


# バッチ推論用関数
def input_fn(input_data, content_type):
    '''入力データを読み込む関数'''
    if content_type == 'text/csv':
        df = pd.read_csv(StringIO(input_data), header=None)
    print(df.shape)
    print(df.head())
    return df


# バッチ推論用関数
def predict_fn(input_data, model):
    '''推論する関数'''
    output = model.predict(input_data) # 入力データと結合する場合はヘッダあり、なしを入力データと統一する
    return output


# モデルの学習（estimator.fit()）をする際はここから実行されます
# 学習データを読み込むパスを引数で受け取るようにすればローカルPCで LightGBM を実行する際と同等の記述ができます
# 最後に学習済みモデルを指定されたパス（model_dir）に保存しています
if __name__ =='__main__':

    print('extracting arguments')
    parser = argparse.ArgumentParser()
    
    # Sagemaker specific arguments. Defaults are set in the environment variables.
    parser.add_argument('--model-dir', type=str, default=os.environ.get('SM_MODEL_DIR'))
    parser.add_argument('--train', type=str, default=os.environ.get('SM_CHANNEL_TRAIN'))
    parser.add_argument('--validation', type=str, default=os.environ.get('SM_CHANNEL_VALIDATION'))
    parser.add_argument('--model-name', type=str, default='lgb-model')
    
    # CustomFramework specific arguments.
    parser.add_argument('--categorical', type=str, default=None)  # in this script we ask user to explicitly name categorical features
    parser.add_argument('--target', type=str) # in this script we ask user to explicitly name the target
    
    # LightGBM specific arguments. Defaults are set in LightGBM default values.
    parser.add_argument('--device_type', type=str, default='cpu')
    parser.add_argument('--seed', type=int, default=None)    
    parser.add_argument('--objective', type=str, default='regression')
    parser.add_argument('--metric', type=str, default='')
    parser.add_argument('--boosting', type=str, default='gbdt')
    parser.add_argument('--num-class', type=int, default=1)
    parser.add_argument('--num-iterations', type=int, default=100)
    parser.add_argument('--early-stopping-rounds', type=int, default=None)
    parser.add_argument('--learning-rate', type=float, default=0.1)
    parser.add_argument('--max_depth', type=int, default=-1)
    parser.add_argument('--num_leaves', type=int, default=31)
    parser.add_argument('--max_bin', type=int, default=255)
    parser.add_argument('--bin_construct_sample_cnt', type=int, default=200000)
    parser.add_argument('--bagging_fraction', type=float, default=1.0)
    parser.add_argument('--bagging_freq', type=int, default=0)
    parser.add_argument('--feature_fraction', type=float, default=1.0)
    parser.add_argument('--lambda_l1', type=float, default=0.0)
    parser.add_argument('--lambda_l2', type=float, default=0.0)
    parser.add_argument('--min_gain_to_split', type=float, default=0.0)
    parser.add_argument('--min_sum_hessian_in_leaf', type=float, default=1e-3)
    parser.add_argument('--min_data_in_leaf', type=int, default=20)
    parser.add_argument('--scale_pos_weight', type=float, default=1.0)
    
    args, _ = parser.parse_known_args()
    
    params = vars(args)
    exclude_param = ['model_dir', 'train', 'validation', 'categorical', 'target', 'model_name']
    params = {k : v for k, v in params.items() if k not in exclude_param}
    
    logger = logging.getLogger()
    logger.setLevel(logging.INFO)
    logging.info('Reading data and building dataset')
    
    if args.categorical != None:
        categorical = args.categorical.split(', ')
    else:
        categorical = "auto"
    logging.info('Set categorical_feature: {}'.format(categorical))
    
    # read train data
    input_files = [os.path.join(args.train, file) for file in os.listdir(args.train)]
    raw_data = [pd.read_csv(file, engine="python") for file in input_files]
    train_df = pd.concat(raw_data)
    X_train = train_df[[x for x in train_df.columns if x not in [args.target]]]
    y_train = train_df[args.target]
    d_train = lgb.Dataset(X_train, label=y_train, feature_name='auto', categorical_feature=categorical)
    
    # read validation data
    input_files = [os.path.join(args.validation, file) for file in os.listdir(args.validation)]
    if len(input_files) != 0:
        raw_data = [pd.read_csv(file, engine="python") for file in input_files]
        val_df = pd.concat(raw_data)
        X_val = val_df[[x for x in val_df.columns if x not in [args.target]]]
        y_val = val_df[args.target]
        d_val = lgb.Dataset(X_val, label=y_val, reference=d_train, feature_name='auto', categorical_feature=categorical)
    else:
        d_val = None
    
    valid_sets = [d_train, d_val] if d_val is not None else [d_train]
    valid_names = ['train', 'valid'] if d_val is not None else ['train']
        
    logging.info('Run LightGBM')
    
    # train LightGBM
    model = lgb.train(params,
                      train_set=d_train,
                      valid_sets=valid_sets,
                      valid_names=valid_names,
                      fobj=None,
                      feval=None,
                      init_model=None,
                      evals_result=None,
                      verbose_eval=True,
                      keep_training_booster=False,
                      callbacks=None,
                     )
    
    # Save model
    output_path = os.path.join(args.model_dir, args.model_name)
    model.save_model(output_path, num_iteration=model.best_iteration)
    logging.info("Stored trained model at {}".format(output_path))

In [ ]:
%%writefile ./src_lgb/requirements.txt
lightgbm==3.3.2

ここでは、XGBoostのコンテナを利用してLightGBMを実行します。    
以下に、カスタムコンテナを作成するサンプルもありますが、XGBoostコンテナを使用する方法が最も簡単です。


Amazon SageMaker におけるカスタムコンテナ実装パターン詳説 〜学習編〜
- https://aws.amazon.com/jp/blogs/news/sagemaker-custom-containers-pattern-training/

BYO Container Example: lightGBM    
- https://github.com/aws-samples/amazon-sagemaker-script-mode/blob/master/lightgbm-byo/lightgbm-byo.ipynb

custom-training-containers    
- https://github.com/aws/amazon-sagemaker-examples/tree/main/advanced_functionality/custom-training-containers

In [ ]:
from sagemaker.xgboost.estimator import XGBoost

output_path = 's3://{}/{}/{}'.format(bucket_name, 'model', ver)
print(output_path)

lgb_estimator = XGBoost(
    entry_point='lightgbm_train.py',
    source_dir='src_lgb', # このdirのrequirements.txtに追加したいライプラリ（LightGBM）を記載
    framework_version='1.3-1',
    role=role,
    instance_type='ml.m5.xlarge', # データのサイズやアルゴリズムによって変更する
    instance_count=1, 
    base_job_name=f'lightgbm-training-{user_name}',
    code_location=output_path,
    output_path=output_path,
    hyperparameters={
        'target': 'income',
        'categorical': ', '.join(categorical_columns),
        'objective': 'binary',
        'metric': 'auc',
    }
)

In [ ]:
lgb_estimator.fit({'train': train_input, 'validation': validation_input})

## 7.モデルの解釈

ここでは [SHAP](https://github.com/slundberg/shap) を利用して、SageMaker で学習したモデルの解釈・分析を行います。

**このセクションではSageMakerの機能は使用していません。pythonのライブラリを使用しています。**

In [ ]:
MODEL_DIR = './model/'
if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)

In [ ]:
from sagemaker.s3 import S3Downloader

S3Downloader.download(
    s3_uri=lgb_estimator.model_data, 
    local_path=MODEL_DIR, 
    sagemaker_session=sagemaker_session
)
# OUTPUT_DIRに解凍します
!tar -zxvf ./model/model.tar.gz -C ./model

In [ ]:
# 学習データ（train_df）の場合、m5.xlargeで40min程度
import shap
import lightgbm as lgb

model_file = 'lgb-model'
booster = lgb.Booster(model_file=os.path.join(MODEL_DIR, model_file))
booster.params["objective"] = "binary:logistic"

explainer = shap.TreeExplainer(booster)
#shap_values = explainer.shap_values(train_df.iloc[:, 1:])
shap_values = explainer.shap_values(val_df.iloc[:, 1:])

In [ ]:
#shap.summary_plot(shap_values, train_df.iloc[:, 1:])
shap.summary_plot(shap_values, val_df.iloc[:, 1:])

## 8.バッチ推論

SageMakerのバッチ変換ジョブを使用してバッチ推論を実行します。

In [ ]:
key = f'dataset/{ver}'
test_prefix = 'test'

boto3.Session().resource('s3').Bucket(bucket_name).Object(f'{key}/{test_prefix}.csv').upload_file(f'{key}/{test_prefix}.csv')
s3_input_test = f's3://{bucket_name}/{key}/{test_prefix}.csv'
print('Done writing to {}'.format(s3_input_test))

In [ ]:
# The location to store the results of the batch transform job
batch_output_path = 's3://{}/{}/{}/{}'.format(bucket_name, 'batch-inference', ver, 'test')
print(batch_output_path)
model_name=f'lightgbm-{user_name}-{ver}'
print(model_name)

transformer = lgb_estimator.transformer(
    instance_count=1, 
    instance_type='ml.m5.xlarge', 
    assemble_with = 'Line', 
    accept = 'text/csv',
    output_path=batch_output_path,
    max_payload=30,
    model_name=model_name,
)

In [ ]:
# start a transform job
transformer.transform(
    data=s3_input_test, 
    split_type='Line', 
    content_type='text/csv',
    input_filter='$[1:]', # この例ではIncome列を除く（入力ファイルにID列など推論に不要となる列がある時に利用できます）
    join_source='Input', 
    output_filter='$[0, -1]', # label, predict
    logs=True
)

transformer.wait()

In [ ]:
# 推論結果のダウンロード
S3Downloader.download(
    s3_uri=batch_output_path, 
    local_path='.', 
    sagemaker_session=sagemaker_session
)

In [ ]:
output_df = pd.read_csv('test.csv.out', sep=",", header=None)
output_df.head(-10)   

In [ ]:
from sklearn.metrics import roc_auc_score

print('Scikit-learn AUC: ', roc_auc_score(output_df[0], output_df[1]))

## 9. モデルの評価、閾値の調整

「7.モデルの解釈」でダウンロードした学習済みモデルを使って性能評価値の確認とより良い閾値の探索を行います。

テスト用データを読み込んで推論を実行します。

In [ ]:
test_data = pd.read_csv(f'{key}/{test_prefix}.csv', header=None).drop(0, axis=1)
test_data_y = pd.read_csv(f'{key}/{test_prefix}.csv', header=None)[0]
output = booster.predict(test_data)

In [ ]:
from sklearn.metrics import roc_auc_score
print('Scikit-learn AUC: ', roc_auc_score(test_data_y, output))

混同行列を作成します。

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

cm = confusion_matrix(test_data_y, output>0.2)
sns.heatmap(cm, annot=True, cmap='Blues')

閾値を 0.5 に設定して各種メトリクスの値を確認してみます。

In [ ]:
print(classification_report(test_data_y, output>0.5))

[こちらのサンプルノートブック](https://github.com/aws-samples/aws-ml-jp/blob/main/sagemaker/xgboost-customer-churn/xgboost_customer_churn.ipynb) の「5-3.最適な閾値を探す」の方法で最適な閾値を探してみます。

In [ ]:
cutoffs = np.arange(0.01, 1, 0.01)
costs = []

for c in cutoffs:
    _predictions = pd.Categorical(np.where(output > c, 1, 0), categories=[0, 1])
    matrix_a = np.array([[0, 100], [500, 100]])
    matrix_b = pd.crosstab(index=test_data_y, columns=_predictions, dropna=False)
    costs.append(np.sum(np.sum(matrix_a * matrix_b)))

costs = np.array(costs)
plt.plot(cutoffs, costs)
plt.show()
print('Cost is minimized near a cutoff of:', cutoffs[np.argmin(costs)], 'for a cost of:', np.min(costs))